# Pedestal inspection

This notebook can be used to inspect the contents of a pedestal analysis

In [ ]:
!ls /eos/cms/store/group/dpg_hgcal/tb_hgcal/2024/hgcalrd/SepTB2024/calibrations/

In [ ]:
#declare global variables, prepare output and import what is needed
import getpass
import json
import os
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit 
import numpy as np
import pandas as pd

_relay = 1726225188
#prepare output for plots
whoami=getpass.getuser()
_outdir=f'/eos/user/{whoami[0]}/{whoami}/www/HGCal/TB2024/Pedestals'
os.system(f'mkdir -p {_outdir}')

## Data I/O

The following defines a function that collates the results from the pedestal and pedestal closure analyses with the cell areas previously computed from geometry principles.

In [ ]:
def getPedestalDataFrame(relay : int = 1726225188, 
                         basedir : str = '/eos/cms/store/group/dpg_hgcal/tb_hgcal/2024/hgcalrd/SepTB2024/calibrations/') -> pd.DataFrame:
    
    #check that files exist
    rawurl = f'{basedir}/Relay{relay}/pedestals.json'
    if not os.path.isfile(rawurl):
        raise ValueError(f'Could not find pedestals file for relay {relay}')

    url = f'{basedir}/Relay{relay}/pedestalsclosure.json'
    if not os.path.isfile(url):
        raise ValueError(f'Could not find pedestals closure file for relay {relay}')
    
 
    #cell area information
    with open('../data/cellareas.json') as fin:
        cellareas = json.load(fin)
    def _assignCellArea(row):    
        typecodeNoBP=row['Typecode'][0:4].replace('-','_')
        return cellareas[typecodeNoBP]['SF']
    
    #read pedestal json
    with open(rawurl) as fin:
        data = pd.read_json(fin,orient='index')
        cols = data.columns
        data.reset_index(inplace=True)
        data = data.rename({'index':'Typecode'},axis=1)
        data['AreaSF'] = data.apply(_assignCellArea, axis=1)
        data['Typecode'] = data['Typecode'].str.replace('-','_')
        data = data.explode(list(cols)+['AreaSF'])
        data['ROC'] = data['Channel']/74
        data = data.astype({'ROC':int})
    
    with open(url) as fin:
        closuredata = pd.read_json(fin,orient='index')
        cols = closuredata.columns
        closuredata.reset_index(inplace=True)
        closuredata = closuredata.rename({'index':'Typecode'},axis=1)
        closuredata['Typecode'] = closuredata['Typecode'].str.replace('-','_')
        def _addChannel(row):
            nch=len(row['En_rms'])
            return [i for i in range(nch)]
        closuredata['Channel'] = closuredata.apply(_addChannel,axis=1)
        closuredata = closuredata.explode(list(cols)+['Channel'])            
    data = data.merge(closuredata, on=['Typecode','Channel'], how='inner')
    
    return data

df = getPedestalDataFrame(relay = _relay)
df.head()

## Check for invalid channels

Simple sanity check on the dataframe to be used for analysis below

In [ ]:
#inspect valid channels
mask=(df['Valid']==0)
print(f'Percentage of invalid channels in relay {_relay}: {100*mask.sum()/df.shape[0]} %')
if mask.sum():
    print(df[mask][['Typecode','Channel']])
else:
    print('\t...yay!')

## Histograms of pedestal-related quantities

Below a series of histograms are drawn for the run of interest.

In [ ]:
def showPlotComparisonsFor(df,var,varLabel,bins,outfile):
    
    if not var in df.columns: return
    plt.style.use(hep.style.CMS)
    
    fig, ax = plt.subplots(1,3,figsize=(24,8))
    
    kwargs={'histtype':'fill','edgecolor':'black','alpha':0.5,'stack':False,'lw':2}
    
    #plot by cell area
    histos=[]
    labels=[]
    for alims in [(0,0.6),(0.6,0.9),(0.9,1.5)]:                
        mask = (df['Valid']==1) & (df['AreaSF']>alims[0]) & (df['AreaSF']<alims[1])
        m=df[mask][var].mean()
        histos.append( np.histogram(df[mask][var],bins=bins)[0] )
        labels.append(rf'${alims[0]:3.1f}<A/A_{{full}}<{alims[1]:3.1f}$ : {m:3.2f}')        
    hep.histplot(histos,bins=bins,**kwargs,ax=ax[0],label=labels)
    
    #plot by module type
    mask = (df['Valid']==True) & (df['AreaSF']>0)
    histos=[]
    labels=[]
    for mod, group in df[mask].groupby('Typecode'):
        m=group[var].mean()
        histos.append( np.histogram(group[var],bins=bins)[0] )
        labels.append(f'{mod} : {m:3.2f}')
    hep.histplot(histos,bins=bins,**kwargs,ax=ax[1],label=labels)
            
    #plot by ROC
    histos=[]
    labels=[]
    for roc, group in df[mask].groupby('ROC'):
        m=group[var].mean()
        histos.append( np.histogram(group[var],bins=bins)[0] )
        labels.append(f'ROC {roc}: {m:3.2f}')
    hep.histplot(histos,bins=bins,**kwargs,ax=ax[2],label=labels)
                  
    for i in range(3):
        ax[i].legend(fontsize=16,loc='upper right')
        ax[i].grid()
        ax[i].set_xlabel(varLabel)
    hep.cms.label(loc=0, exp='HGCAL', llabel='', rlabel="Sep'24",ax=ax[0])
    fig.tight_layout()
    plt.savefig(f'{outfile}_{var}.png')
    plt.close()

for var, varLabel, bins in [
    ('cm2_slope',  r'ADC vs $CM_{2}$ slope',   np.linspace(-0.2,1.0,50)),
    ('cm4_slope',  r'ADC vs $CM_{4}$ slope',   np.linspace(-0.2,1.0,50)),
    ('cmall_slope',r'ADC vs $CM_{all}$ slope', np.linspace(-0.2,1.0,50)),
    ('adcm1_slope',r'ADC vs ADC(BX-1) slope', np.linspace(-0.2,0.2,50)),                            
    ('ADC_rms','RMS(ADC)',np.linspace(0,5,50)),
    ('ADC_ped','Pedestal',np.linspace(50.5,200.5,151)),
    ('En_ped','Residual pedestal',np.linspace(-5,5,50)),
    ('En_rms','RMS(RecHit)',np.linspace(0,5,50)),
    ('CM_slope','Residual $CM_{2}$ slope',np.linspace(-0.2,1,50)),
    ('CNF','Coeherent noise fraction',np.linspace(0,0.5,50)),
]:
    showPlotComparisonsFor(df, var,varLabel,bins, f'{_outdir}/Relay{_relay}')

In [ ]:
def simplePlotComparisonsFor(df,var,varLabel,bins,outfile):
    
    if not var in df.columns: return
    plt.style.use(hep.style.CMS)
    
    fig, ax = plt.subplots(figsize=(10,10))    
    ax.hist(df[var],bins=bins,histtype='step',lw=2)
    hep.cms.label(loc=0, exp='HGCAL', llabel='', rlabel="Sep'24",ax=ax)
    ax.grid()
    ax.set_xlabel(varLabel)
    ax.set_ylabel('Channels')
    fig.tight_layout()
    plt.savefig(f'{outfile}_{var}.png')
    plt.close()

for var, varLabel, bins in [
    ('ADC_ped','Pedestal',np.linspace(50.5,200.5,151)),
    ('En_ped','Residual pedestal',np.linspace(-5,5,50)),
]:
    simplePlotComparisonsFor(df, var,varLabel,bins, f'{_outdir}/Relay{_relay}_inc')

In [ ]:
import ROOT
import itertools

def showCMCorrelation( erx : int,
                       relay : int,
                       outfile : str,
                       xran = None,
                       yran = None,
                       basedir : str = '/eos/cms/store/group/dpg_hgcal/tb_hgcal/2024/hgcalrd/SepTB2024/calibrations/',
                       module : str = 'ML_F3WX_IH0018'):
    
    url = f'{basedir}/Relay{relay}/histofiller/{module}.root'
    fIn = ROOT.TFile.Open(url)
    h = fIn.Get('adcvscm2')
    nx, xmin, xmax = h.GetNbinsX(), h.GetXaxis().GetXmin(), h.GetXaxis().GetXmax()
    ny, ymin, ymax = h.GetNbinsY(), h.GetYaxis().GetXmin(), h.GetYaxis().GetXmax()
    nz, zmin, zmax = h.GetNbinsZ(), h.GetZaxis().GetXmin(), h.GetZaxis().GetXmax()
    cmvsadc = ROOT.TH2F('cmvsadc',';Common mode;ADC;Events',ny,ymin,ymax,nz,zmin,zmax)

    ROOT.gStyle.SetOptStat(0)
    ROOT.gStyle.SetOptFit(0)
    c = ROOT.TCanvas('c','c',1200,1200)
    c.Divide(5,8)

    histos = []
    labels = []
    for i in range(37):
        ch = i + erx*37

        p = c.cd(i+1)
        p.SetGridx()
        p.SetGridy()
        p.SetTopMargin(0)
        p.SetLeftMargin(0.15 if i%5==0 else 0)
        p.SetRightMargin(0)
        p.SetBottomMargin(0.16 if i>34 else 0)
        histos.append( cmvsadc.Clone(f'ch_{ch}') )
        for iy,iz in itertools.product( range(ny), range(nz) ):
            cts = h.GetBinContent(ch+1,iy+1,iz+1)
            histos[-1].SetBinContent(iy+1,iz+1,cts)
        histos[-1].Draw('col')
        histos[-1].GetXaxis().SetTitleSize(0.1)
        histos[-1].GetYaxis().SetTitleSize(0.1)
        histos[-1].GetXaxis().SetTitleOffset(0.7)
        histos[-1].GetYaxis().SetTitleOffset(0.8)
        histos[-1].GetXaxis().SetLabelSize(0.1)
        histos[-1].GetYaxis().SetLabelSize(0.1)
        
        histos.append( histos[-1].ProfileX() )
        histos[-1].SetMarkerStyle(20)
        histos[-1].SetMarkerSize(0.05)
        histos[-1].SetMarkerColor(1)
        histos[-1].Draw('e1same')
        histos[-1].Fit('pol1','QM+','same')
        
        labels.append( ROOT.TLatex() )
        labels[-1].SetNDC()
        labels[-1].SetTextFont(42)
        labels[-1].SetTextSize(0.1)
        labels[-1].DrawLatex(0.2 if i%5==0 else 0.1,0.9,f'Channel {ch}')

        if not xran is None:
            histos[-2].GetXaxis().SetRangeUser(*xran)
        if not yran is None:
            histos[-2].GetYaxis().SetRangeUser(*yran)
        
    c.cd(40)
    txt  = ROOT.TLatex()
    txt.SetNDC()
    txt.SetTextFont(42)
    txt.SetTextSize(0.15)
    txt.DrawLatex(0.2,0.8,"#bf{HGCAL} Sep'2024")
    txt.DrawLatex(0.2,0.5,f'Relay {relay}')
    txt.DrawLatex(0.2,0.2,f'e-Rx {erx}')
    
    c.cd()
    c.Modified()
    c.Update()
    c.SaveAs(f'{outfile}_erx{erx}.png')
    fIn.Close()

for erx in range(6):
    showCMCorrelation(erx,_relay,f'{_outdir}/Relay{_relay}_cm') #,xran=(80,120),yran=(80,220))

# Noise modelling

In the absence of leakage current (sensors have not yet been irradiated), the noise is expected to scale with the capacitance, and hence with the area of the cells.
In the following we assume a simple quadratic scaling (as indicated by HGCROC simulations) and fit it to the data.

In [ ]:
def _noiseModel(x,a,b,c): #,k):
    """assumes that the noise scales quadratically with the capacitance (or the cell area)"""
    cap=x
    #cap=x[:,0]
    #trace=x[:,1]
    return (a*(cap**2)+b*cap+c) #*(1+k*trace)

def runNoiseModelFit(df,var,outfile):

    mask = (df['Valid']==1) & (df['AreaSF']>0)
    x=df[mask]['AreaSF'].values.astype(float)
    xs=np.linspace(0.1,1.3,100)
    y=df[mask][var].values.astype(float)
    popt, pcov = curve_fit(_noiseModel, x, y, bounds=([0,0,0], [2., 2., 5.]))
    yvars=[]
    for i in range(len(popt)):
        popt_up=popt.copy()
        popt_up[i]=popt_up[i]+np.sqrt(pcov[i][i])
        yvars.append(_noiseModel(xs,*popt_up))

        popt_dn=popt.copy()
        popt_dn[i]=popt_dn[i]-np.sqrt(pcov[i][i])
        yvars.append(_noiseModel(xs,*popt_dn))
    yvars=np.array(yvars)

    #show plot
    fig, ax = plt.subplots(figsize=(10,10))

    #expected
    #typical adc2fC from CalPulse : NOTE here we could simply read the calpulse file
    adc2fC=0.19 
    Cap=xs*45.  #capacitance of 300 um
    exp_param=(0.000017*(Cap**2)+0.002119*Cap+0.190295)/adc2fC

    #observed
    H, xedges, yedges = np.histogram2d(x, y, bins=(np.linspace(0.1,1.5,40), np.linspace(0,5,50)))
    hep.hist2dplot(H, xedges, yedges, labels=False,cmin=1)
    ax.fill_between(xs,yvars.min(axis=0), yvars.max(axis=0),alpha=0.2)
    ax.plot(xs,_noiseModel(xs,*popt),lw=2,ls='-',c='r',
            label=rf'${popt[0]:3.2f} \cdot \left( \frac{{A}}{{A_{{full}}}}  \right)^2 + {popt[1]:3.3f} \cdot \left( \frac{{A}}{{A_{{full}}}}  \right) + {popt[0]:3.3f}$')

    #expected
    ax.plot(xs,exp_param, lw=2, ls='--', c='gray',label='HGCROC simulation')

    ax.legend(loc='upper left')
    ax.grid()
    ax.set_ylabel('Noise')
    ax.set_ylim(0.5,4)
    ax.set_xlabel(r'$\frac{{A}}{{A_{{full}}}}$ = Cell area / Full cell area')
    hep.cms.label(loc=0, exp='HGCAL', llabel='', rlabel="Sep'24",ax=ax)
    fig.tight_layout()
    plt.savefig(outfile)
    plt.close()
    
for var in ['ADC_rms','En_rms']:
    runNoiseModelFit(df,var,f'{_outdir}/Relay{_relay}_{var}_noisescaling')